# Part I: Data Curation 

## Perform ETL and Data Cleaning

In [188]:
import pandas as pd 
import requests 
import numpy as np

churn_data_df = pd.read_excel("Telco_churn.xlsx")
churn_data_df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


##  Drop unneccessary columns

In [189]:
churn_data_df = churn_data_df.drop(["CustomerID","City","Count","Country","State","Zip Code", "Lat Long", "Latitude","Longitude"], axis = 1)
churn_data_df = churn_data_df.drop(["Churn Reason","CLTV","Churn Score","Churn Label"], axis = 1)
churn_data_df = churn_data_df.drop(["Internet Service","Contract","Payment Method", "Paperless Billing"], axis = 1)
churn_data_df

,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Monthly Charges,Total Charges,Churn Value
0,Male,No,No,No,2,Yes,No,Yes,Yes,No,No,No,No,53.85,108.15,1
1,Female,No,No,Yes,2,Yes,No,No,No,No,No,No,No,70.70,151.65,1
2,Female,No,No,Yes,8,Yes,Yes,No,No,Yes,No,Yes,Yes,99.65,820.5,1
3,Female,No,Yes,Yes,28,Yes,Yes,No,No,Yes,Yes,Yes,Yes,104.80,3046.05,1
4,Male,No,No,Yes,49,Yes,Yes,No,Yes,Yes,No,Yes,Yes,103.70,5036.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,No,No,No,72,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,21.15,1419.4,0
7039,Male,No,Yes,Yes,24,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,84.80,1990.5,0
7040,Female,No,Yes,Yes,72,Yes,Yes,No,Yes,Yes,No,Yes,Yes,103.20,7362.9,0
7041,Female,No,Yes,Yes,11,No,No phone service,Yes,No,No,No,No,No,29.60,346.45,0


## Perform Duplication checks

In [190]:
# Check
number_of_duplicates = churn_data_df.duplicated().sum()
print (f" Number of duplicates before : {number_of_duplicates}")

# Drop dupes
churn_data_df = churn_data_df.drop_duplicates()

# Check again
number_of_duplicates = churn_data_df.duplicated().sum()
print (f" Number of duplicates after removing : {number_of_duplicates}")

 Number of duplicates before : 51
 Number of duplicates after removing : 0


In [191]:
churn_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6992 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             6992 non-null   object 
 1   Senior Citizen     6992 non-null   object 
 2   Partner            6992 non-null   object 
 3   Dependents         6992 non-null   object 
 4   Tenure Months      6992 non-null   int64  
 5   Phone Service      6992 non-null   object 
 6   Multiple Lines     6992 non-null   object 
 7   Online Security    6992 non-null   object 
 8   Online Backup      6992 non-null   object 
 9   Device Protection  6992 non-null   object 
 10  Tech Support       6992 non-null   object 
 11  Streaming TV       6992 non-null   object 
 12  Streaming Movies   6992 non-null   object 
 13  Monthly Charges    6992 non-null   float64
 14  Total Charges      6992 non-null   object 
 15  Churn Value        6992 non-null   int64  
dtypes: float64(1), int64(2),

## Featurize Columns

In [192]:
def featurize(df):
    X = df[['Tenure Months','Monthly Charges','Total Charges','Churn Value']]
    X['Gender'] = [1 if x=='Male' else 0 for x in df['Gender']] 
    X['Senior Citizen'] = [1 if x=='Yes' else 0 for x in df['Senior Citizen']] 
    X['Partner'] = [1 if x=='Yes' else 0 for x in df['Partner']] 
    X['Dependents'] = [1 if x=='Yes' else 0 for x in df['Dependents']] 
    X['Phone Service'] = [1 if x=='Yes' else 0 for x in df['Phone Service']]
    groups = {
        'Yes':1, 'No phone service': None, 'No':0
    }
    X['Multiple Lines'] = [groups.get(x) for x in df['Multiple Lines']]
    groups = {
        'Yes':1, 'No internet service': None, 'No':0
    }
    X['Online Security'] = [groups.get(x) for x in df['Online Security']]
    groups = {
        'Yes':1, 'No internet service': None, 'No':0
    }
    X['Online Backup'] = [groups.get(x) for x in df['Online Backup']]
    groups = {
        'Yes':1, 'No internet service': None, 'No':0
    }
    X['Device Protection'] = [groups.get(x) for x in df['Device Protection']]
    groups = {
        'Yes':1, 'No internet service': None, 'No':0
    }
    X['Tech Support'] = [groups.get(x) for x in df['Tech Support']]
    groups = {
        'Yes':1, 'No internet service': None, 'No':0
    }
    X['Streaming TV'] = [groups.get(x) for x in df['Streaming TV']]
    groups = {
        'Yes':1, 'No internet service': None, 'No':0
    }
    X['Streaming Movies'] = [groups.get(x) for x in df['Streaming Movies']]
    
    return X

churn_data_df = featurize(churn_data_df)
churn_data_df

C:\Users\Zach Soo\AppData\Local\Temp\ipykernel_30604\2465787286.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Gender'] = [1 if x=='Male' else 0 for x in df['Gender']]
C:\Users\Zach Soo\AppData\Local\Temp\ipykernel_30604\2465787286.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Senior Citizen'] = [1 if x=='Yes' else 0 for x in df['Senior Citizen']]
C:\Users\Zach Soo\AppData\Local\Temp\ipykernel_30604\2465787286.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,Tenure Months,Monthly Charges,Total Charges,Churn Value,Gender,Senior Citizen,Partner,Dependents,Phone Service,Multiple Lines,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies
0,2,53.85,108.15,1,1,0,0,0,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,2,70.70,151.65,1,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,99.65,820.5,1,0,0,0,1,1,1.0,0.0,0.0,1.0,0.0,1.0,1.0
3,28,104.80,3046.05,1,0,0,1,1,1,1.0,0.0,0.0,1.0,1.0,1.0,1.0
4,49,103.70,5036.3,1,1,0,0,1,1,1.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,72,21.15,1419.4,0,0,0,0,0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7039,24,84.80,1990.5,0,1,0,1,1,1,1.0,1.0,0.0,1.0,1.0,1.0,1.0
7040,72,103.20,7362.9,0,0,0,1,1,1,1.0,0.0,1.0,1.0,0.0,1.0,1.0
7041,11,29.60,346.45,0,0,0,1,1,0,NaN,1.0,0.0,0.0,0.0,0.0,0.0


## Drop Null values

In [193]:
# Drop null
churn_data_df = churn_data_df.dropna()

# Check again
churn_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4824 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tenure Months      4824 non-null   int64  
 1   Monthly Charges    4824 non-null   float64
 2   Total Charges      4824 non-null   object 
 3   Churn Value        4824 non-null   int64  
 4   Gender             4824 non-null   int64  
 5   Senior Citizen     4824 non-null   int64  
 6   Partner            4824 non-null   int64  
 7   Dependents         4824 non-null   int64  
 8   Phone Service      4824 non-null   int64  
 9   Multiple Lines     4824 non-null   float64
 10  Online Security    4824 non-null   float64
 11  Online Backup      4824 non-null   float64
 12  Device Protection  4824 non-null   float64
 13  Tech Support       4824 non-null   float64
 14  Streaming TV       4824 non-null   float64
 15  Streaming Movies   4824 non-null   float64
dtypes: float64(8), int64(7),

In [194]:
churn_data_df

,Tenure Months,Monthly Charges,Total Charges,Churn Value,Gender,Senior Citizen,Partner,Dependents,Phone Service,Multiple Lines,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies
0,2,53.85,108.15,1,1,0,0,0,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,2,70.70,151.65,1,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,99.65,820.5,1,0,0,0,1,1,1.0,0.0,0.0,1.0,0.0,1.0,1.0
3,28,104.80,3046.05,1,0,0,1,1,1,1.0,0.0,0.0,1.0,1.0,1.0,1.0
4,49,103.70,5036.3,1,1,0,0,1,1,1.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7035,38,69.50,2625.25,0,1,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7036,19,78.70,1495.1,0,1,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7039,24,84.80,1990.5,0,1,0,1,1,1,1.0,1.0,0.0,1.0,1.0,1.0,1.0
7040,72,103.20,7362.9,0,0,0,1,1,1,1.0,0.0,1.0,1.0,0.0,1.0,1.0


## Convert floats in columns to int

In [195]:
def convert_float_to_int(f):
    converted = int(f)
    return converted

churn_data_df["Multiple Lines"] = churn_data_df["Multiple Lines"].apply(convert_float_to_int)
churn_data_df["Online Security"] = churn_data_df["Online Security"].apply(convert_float_to_int)
churn_data_df["Online Backup"] = churn_data_df["Online Backup"].apply(convert_float_to_int)
churn_data_df["Device Protection"] = churn_data_df["Device Protection"].apply(convert_float_to_int)
churn_data_df["Tech Support"] = churn_data_df["Tech Support"].apply(convert_float_to_int)
churn_data_df["Streaming TV"] = churn_data_df["Streaming TV"].apply(convert_float_to_int)
churn_data_df["Streaming Movies"] = churn_data_df["Streaming Movies"].apply(convert_float_to_int)

churn_data_df

C:\Users\Zach Soo\AppData\Local\Temp\ipykernel_30604\1403353597.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  churn_data_df["Multiple Lines"] = churn_data_df["Multiple Lines"].apply(convert_float_to_int)
C:\Users\Zach Soo\AppData\Local\Temp\ipykernel_30604\1403353597.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  churn_data_df["Online Security"] = churn_data_df["Online Security"].apply(convert_float_to_int)
C:\Users\Zach Soo\AppData\Local\Temp\ipykernel_30604\1403353597.py:7: SettingWithCopyWarni

,Tenure Months,Monthly Charges,Total Charges,Churn Value,Gender,Senior Citizen,Partner,Dependents,Phone Service,Multiple Lines,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies
0,2,53.85,108.15,1,1,0,0,0,1,0,1,1,0,0,0,0
1,2,70.70,151.65,1,0,0,0,1,1,0,0,0,0,0,0,0
2,8,99.65,820.5,1,0,0,0,1,1,1,0,0,1,0,1,1
3,28,104.80,3046.05,1,0,0,1,1,1,1,0,0,1,1,1,1
4,49,103.70,5036.3,1,1,0,0,1,1,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7035,38,69.50,2625.25,0,1,0,0,0,1,0,0,0,0,0,0,0
7036,19,78.70,1495.1,0,1,0,0,0,1,0,0,0,0,0,1,0
7039,24,84.80,1990.5,0,1,0,1,1,1,1,1,0,1,1,1,1
7040,72,103.20,7362.9,0,0,0,1,1,1,1,0,1,1,0,1,1


## Sanity check for duplicates

In [196]:
# Check
number_of_duplicates = churn_data_df.duplicated().sum()
print (f" Number of duplicates before : {number_of_duplicates}")

# Drop dupes
churn_data_df = churn_data_df.drop_duplicates()

# Check again
number_of_duplicates = churn_data_df.duplicated().sum()
print (f" Number of duplicates after removing : {number_of_duplicates}")

 Number of duplicates before : 0
 Number of duplicates after removing : 0


## Split the Dataset into Test and Training Data for ML Classifier model

In [197]:
from sklearn.model_selection import train_test_split
x = churn_data_df.drop('Churn Value', axis=1)
y = churn_data_df["Churn Value"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [198]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3859 entries, 4492 to 1000
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tenure Months      3859 non-null   int64  
 1   Monthly Charges    3859 non-null   float64
 2   Total Charges      3859 non-null   object 
 3   Gender             3859 non-null   int64  
 4   Senior Citizen     3859 non-null   int64  
 5   Partner            3859 non-null   int64  
 6   Dependents         3859 non-null   int64  
 7   Phone Service      3859 non-null   int64  
 8   Multiple Lines     3859 non-null   int64  
 9   Online Security    3859 non-null   int64  
 10  Online Backup      3859 non-null   int64  
 11  Device Protection  3859 non-null   int64  
 12  Tech Support       3859 non-null   int64  
 13  Streaming TV       3859 non-null   int64  
 14  Streaming Movies   3859 non-null   int64  
dtypes: float64(1), int64(13), object(1)
memory usage: 482.4+ KB


In [199]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 965 entries, 4487 to 5436
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tenure Months      965 non-null    int64  
 1   Monthly Charges    965 non-null    float64
 2   Total Charges      965 non-null    object 
 3   Gender             965 non-null    int64  
 4   Senior Citizen     965 non-null    int64  
 5   Partner            965 non-null    int64  
 6   Dependents         965 non-null    int64  
 7   Phone Service      965 non-null    int64  
 8   Multiple Lines     965 non-null    int64  
 9   Online Security    965 non-null    int64  
 10  Online Backup      965 non-null    int64  
 11  Device Protection  965 non-null    int64  
 12  Tech Support       965 non-null    int64  
 13  Streaming TV       965 non-null    int64  
 14  Streaming Movies   965 non-null    int64  
dtypes: float64(1), int64(13), object(1)
memory usage: 120.6+ KB


In [200]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 3859 entries, 4492 to 1000
Series name: Churn Value
Non-Null Count  Dtype
--------------  -----
3859 non-null   int64
dtypes: int64(1)
memory usage: 60.3 KB


In [201]:
y_test.info()

<class 'pandas.core.series.Series'>
Int64Index: 965 entries, 4487 to 5436
Series name: Churn Value
Non-Null Count  Dtype
--------------  -----
965 non-null    int64
dtypes: int64(1)
memory usage: 15.1 KB


## Save dataframes to csv files

In [202]:
X_train.to_csv("X_train.csv")
X_test.to_csv("X_test.csv")
y_train.to_csv("y_train.csv")
y_test.to_csv("y_test.csv")